In [143]:
import numpy as np
import pandas as pd
import scipy as sp

import json
import bson
from bson.json_util import dumps,default
import uuid

import glob

from pymongo import MongoClient

In [4]:
with open('datasets/squad-train-v1.1.json') as json_file:  
    json_data = json.load(json_file)

In [7]:
data = json_data['data']

In [10]:
print(len(data))

442


In [11]:
item_1 = data[0]
print(item_1)

{'title': 'University_of_Notre_Dame', 'paragraphs': [{'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'qas': [{'answers': [{'answer_start': 515, 'text': 'Saint Bernadette Soubirous'}], 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'id': '5733be284776f41900661182'}, {'answers': [{'answer_start': 

In [13]:
for key in item_1.keys():
    print(key)

title
paragraphs


In [14]:
print(item_1['title'])
paragraphs = item_1['paragraphs']
print(len(paragraphs))
paragraph_1 = paragraphs[0]
for key in paragraph_1.keys():
    print(key)

University_of_Notre_Dame
55
context
qas


In [15]:
context = paragraph_1['context']
print(context)

Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.


In [17]:
qas = paragraph_1['qas']
print(len(qas))
for q in qas:
    print(q)

5
{'answers': [{'answer_start': 515, 'text': 'Saint Bernadette Soubirous'}], 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'id': '5733be284776f41900661182'}
{'answers': [{'answer_start': 188, 'text': 'a copper statue of Christ'}], 'question': 'What is in front of the Notre Dame Main Building?', 'id': '5733be284776f4190066117f'}
{'answers': [{'answer_start': 279, 'text': 'the Main Building'}], 'question': 'The Basilica of the Sacred heart at Notre Dame is beside to which structure?', 'id': '5733be284776f41900661180'}
{'answers': [{'answer_start': 381, 'text': 'a Marian place of prayer and reflection'}], 'question': 'What is the Grotto at Notre Dame?', 'id': '5733be284776f41900661181'}
{'answers': [{'answer_start': 92, 'text': 'a golden statue of the Virgin Mary'}], 'question': 'What sits on top of the Main Building at Notre Dame?', 'id': '5733be284776f4190066117e'}


In [24]:
def get_qas(item,p_count=0):
    print('title:',item['title'])
    paragraphs = item['paragraphs']
    print('# of paragraphs:',len(paragraphs))
    
    paragraph = paragraphs[p_count]
    context = paragraph['context']
    print('context:',context)
    qas = paragraph['qas']
    print('# of qas',len(qas))
    for q in qas:
        print('Question:',q['question'])
        print('Answers:',q['answers'][0])
        print(context[q['answers'][0]['answer_start']])

get_qas(data[0])

title: University_of_Notre_Dame
# of paragraphs: 55
context: Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
# of qas 5
Question: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Answers: {'answer_start': 515, 'text': 'Saint Bernadette Soubirous'}
S
Question: What is in front of the Notre Dame Main Building?
Answers: {'answer_s

In [25]:
get_qas(data[4],p_count=1)

title: Antibiotics
# of paragraphs: 26
context: The successful outcome of antimicrobial therapy with antibacterial compounds depends on several factors. These include host defense mechanisms, the location of infection, and the pharmacokinetic and pharmacodynamic properties of the antibacterial. A bactericidal activity of antibacterials may depend on the bacterial growth phase, and it often requires ongoing metabolic activity and division of bacterial cells. These findings are based on laboratory studies, and in clinical settings have also been shown to eliminate bacterial infection. Since the activity of antibacterials depends frequently on its concentration, in vitro characterization of antibacterial activity commonly includes the determination of the minimum inhibitory concentration and minimum bactericidal concentration of an antibacterial. To predict clinical outcome, the antimicrobial activity of an antibacterial is usually combined with its pharmacokinetic profile, and several ph

In [26]:
get_qas(data[8],p_count=3)

title: The_Legend_of_Zelda:_Twilight_Princess
# of paragraphs: 32
context: The Legend of Zelda: Twilight Princess is an action-adventure game focused on combat, exploration, and item collection. It uses the basic control scheme introduced in Ocarina of Time, including context-sensitive action buttons and L-targeting (Z-targeting on the Wii), a system that allows the player to keep Link's view focused on an enemy or important object while moving and attacking. Link can walk, run, and attack, and will automatically jump when running off of or reaching for a ledge.[c] Link uses a sword and shield in combat, complemented with secondary weapons and items, including a bow and arrows, a boomerang, bombs, and the Clawshot (similar to the Hookshot introduced earlier in the The Legend of Zelda series).[d] While L-targeting, projectile-based weapons can be fired at a target without the need for manual aiming.[c]
# of qas 9
Question: What are the three main activities in The Legend of Zelda: Twili

In [133]:
# add custom rows to data
custom_content = {'title': 'Data_Science','custom':True
                  ,'paragraphs':[
                      {
                          'context': 'Data science, also known as data-driven science, is an interdisciplinary field about scientific methods, processes, and systems to extract knowledge or insights from data in various forms, either structured or unstructured, similar to data mining.'
                          , 'qas': [
                              {
                                  'answers': [{'answer_start': 49, 'text': 'is an interdisciplinary field about scientific methods, processes, and systems to extract knowledge or insights from data'}]
                                  , 'question': 'What is Data science?'
                                  , 'id': uuid.uuid4().hex
                              }
                              ,{
                                  'answers': [{'answer_start': 0, 'text': 'Data science'}]
                                  , 'question': 'What is similar to data mining?'
                                  , 'id': uuid.uuid4().hex
                              }
                          ]
                      }
                      ,{
                          'context':'Machine learning is a field of computer science that gives computers the ability to learn without being explicitly programmed. Arthur Samuel, an American pioneer in the field of computer gaming and artificial intelligence, coined the term "Machine Learning" in 1959 while at IBM.'
                          , 'qas': [
                              {
                                  'answers': [{'answer_start': 17, 'text': 'is a field of computer science that gives computers the ability to learn without being explicitly programmed'}]
                                  , 'question': 'What is Machine learning?'
                                  , 'id': uuid.uuid4().hex
                              }
                              ,{
                                  'answers': [{'answer_start': 128, 'text': 'Arthur Samuel'}]
                                  , 'question': 'Who is a pioneer in the field of computer gaming and artificial intelligence?'
                                  , 'id': uuid.uuid4().hex
                              }
                          ]
                      }
                      
                  ]}

In [134]:
print(custom_content)

{'title': 'Data_Science', 'custom': True, 'paragraphs': [{'context': 'Data science, also known as data-driven science, is an interdisciplinary field about scientific methods, processes, and systems to extract knowledge or insights from data in various forms, either structured or unstructured, similar to data mining.', 'qas': [{'answers': [{'answer_start': 49, 'text': 'is an interdisciplinary field about scientific methods, processes, and systems to extract knowledge or insights from data'}], 'question': 'What is Data science?', 'id': 'baf91df8cc2144b3bfa3e23fc47addc6'}, {'answers': [{'answer_start': 0, 'text': 'Data science'}], 'question': 'What is similar to data mining?', 'id': 'f4e35cdaa5e743d9b64bc67693c65d8c'}]}, {'context': 'Machine learning is a field of computer science that gives computers the ability to learn without being explicitly programmed. Arthur Samuel, an American pioneer in the field of computer gaming and artificial intelligence, coined the term "Machine Learning" i

In [40]:
str(uuid.uuid4())

'e8f26cbc-e5ca-43cf-a31e-f6975050ce0b'

In [41]:
uuid.uuid4().hex

'ba06861d1dad476c8e0a5e21f68eb4d1'

## Load Mongo with the SQuAD data

In [146]:
client = MongoClient('mongodb://localhost:27017')
db = client['squad-db']

In [136]:
with open('datasets/squad-train-v1.1.json') as json_file:  
    json_data = json.load(json_file)
    
data = json_data['data']

# Merge custom content
data.append(custom_content)

In [137]:
get_qas(data[442],p_count=0)

title: Data_Science
# of paragraphs: 2
context: Data science, also known as data-driven science, is an interdisciplinary field about scientific methods, processes, and systems to extract knowledge or insights from data in various forms, either structured or unstructured, similar to data mining.
# of qas 2
Question: What is Data science?
Answers: {'answer_start': 49, 'text': 'is an interdisciplinary field about scientific methods, processes, and systems to extract knowledge or insights from data'}
i
Question: What is similar to data mining?
Answers: {'answer_start': 0, 'text': 'Data science'}
D


In [61]:
# load mongo
dbcollection = db['squad']
dbcollection.drop()
# for item in data:
#     #print(item['title'])
#     result = dbcollection.insert_one(item)

In [ ]:
# Generate json from mongo db collection
result = dbcollection.find({})
output_json = {'data':json.loads(dumps(result))}

In [138]:
# load mongo
titles = db['titles']
titles.drop()
paragraphs = db['paragraphs']
paragraphs.drop()
for title in data:
    result = titles.insert_one({'title':title['title']})
    _id = result.inserted_id
    for paragraph in title['paragraphs']:
        paragraph['title_id'] = str(_id)
        result2 = paragraphs.insert_one(paragraph)

In [153]:
# Generate json from mongo db collection
titles = db['titles']
paragraphs = db['paragraphs']
result = titles.find({})
all_data = []
for title in result:
    paragraphs_data = paragraphs.find({'title_id':str(title['_id'])})
    all_data_item = {}
    all_data_item['title'] = title['title']
    all_data_item['paragraphs'] = paragraphs_data
    all_data.append(all_data_item)

In [154]:
output_json = {'data':json.loads(dumps(all_data))}

In [155]:
# write to json file
with open('../allennlp/training_config/squad-modified.json', 'w') as outfile:  
    json.dump(output_json, outfile)

In [88]:
#print(output_json)

In [101]:
from nltk.corpus import stopwords
cachedStopWords = stopwords.words("english")



hello bye hi


In [103]:
text = 'What is Machine learning?'
text = ' '.join([word for word in text.split() if word not in cachedStopWords])
print(text)

What Machine learning?


### Cosine Similarity


In [115]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import string
import nltk
import re

In [117]:
# Tokenize text using NLTK
def tokenize_text(text,remove_stop_words=True,stem_words=True,filter_short_token=0): # split into words
    words = nltk.word_tokenize(text)
    # convert to lower case
    words = [w.lower() for w in words]
    # prepare regex for char filtering
    re_punc = re.compile('[%s]' % re.escape(string.punctuation)) # remove punctuation from each word
    tokens = [re_punc.sub('', w) for w in words]
    # remove not alphabets
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    if remove_stop_words == True:
        stop_words = set(nltk.corpus.stopwords.words('english'))
        tokens = [w for w in tokens if not w in stop_words]
    # Perfomring
    if stem_words == True:
        # stemming of words
        porter = nltk.stem.porter.PorterStemmer()
        tokens = [porter.stem(word) for word in tokens]
    # filter out short tokens
    if filter_short_token > 0:
        tokens = [word for word in tokens if len(word) > filter_short_token]
    return tokens

In [119]:
corpus = list()
for pg in data[442]['paragraphs']:
    print(pg['context'])
    
    # Get the scene text
    context_text = pg['context']
    # Tokenize
    tokens = tokenize_text(context_text,remove_stop_words=True,stem_words=True,filter_short_token=1)
    cleaned_context_text = ' '.join(tokens)
    
    corpus.append(cleaned_context_text)

Data science, also known as data-driven science, is an interdisciplinary field about scientific methods, processes, and systems to extract knowledge or insights from data in various forms, either structured or unstructured, similar to data mining.
Machine learning is a field of computer science that gives computers the ability to learn without being explicitly programmed. Arthur Samuel, an American pioneer in the field of computer gaming and artificial intelligence, coined the term "Machine Learning" in 1959 while at IBM.


In [123]:
# Tf–idf term weighting using TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)
feature_array = np.array(tfidf_vectorizer.get_feature_names())
print("Bag of Words:",feature_array.shape)
print(feature_array)
print("TF-IDF Matrix:",tfidf_matrix.shape)
tfidf_array = tfidf_matrix.toarray()
print(tfidf_array)

Bag of Words: (39,)
['abil' 'also' 'american' 'arthur' 'artifici' 'coin' 'comput' 'data'
 'datadriven' 'either' 'explicitli' 'extract' 'field' 'form' 'game' 'give'
 'ibm' 'insight' 'intellig' 'interdisciplinari' 'knowledg' 'known' 'learn'
 'machin' 'method' 'mine' 'pioneer' 'process' 'program' 'samuel' 'scienc'
 'scientif' 'similar' 'structur' 'system' 'term' 'unstructur' 'variou'
 'without']
TF-IDF Matrix: (2, 39)
[[ 0.          0.18401758  0.          0.          0.          0.          0.
   0.55205273  0.18401758  0.18401758  0.          0.18401758  0.13093002
   0.18401758  0.          0.          0.          0.18401758  0.
   0.18401758  0.18401758  0.18401758  0.          0.          0.18401758
   0.18401758  0.          0.18401758  0.          0.          0.26186004
   0.18401758  0.18401758  0.18401758  0.18401758  0.          0.18401758
   0.18401758  0.        ]
 [ 0.15904862  0.          0.15904862  0.15904862  0.15904862  0.15904862
   0.47714585  0.          0.          0

In [129]:
question_text = "what is data science?"
# Tokenize
tokens = tokenize_text(question_text,remove_stop_words=True,stem_words=True,filter_short_token=1)
cleaned_context_text = ' '.join(tokens)
question_vector = tfidf_vectorizer.transform([cleaned_context_text])
question_array = question_vector.toarray()
print(question_array)

[[ 0.          0.          0.          0.          0.          0.          0.
   0.81480247  0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.57973867  0.          0.          0.          0.
   0.          0.          0.          0.        ]]


In [130]:
# Find Cosine Similarity of question with all the contexts
cs = cosine_similarity(question_vector, tfidf_matrix)

In [131]:
cs

array([[ 0.60162432,  0.06560578]])

### Read Trained Models

In [144]:
print(glob.glob("../allennlp/train_out/*.gz"))

['../allennlp/train_out/model02.tar.gz', '../allennlp/train_out/model04.tar.gz', '../allennlp/train_out/model01.tar.gz', '../allennlp/train_out/model03.tar.gz', '../allennlp/train_out/bidaf-model-2017.09.15-charpad.tar.gz']


In [145]:
import os
models = os.listdir("../allennlp/train_out")
print(models)

['.DS_Store', 'model02.tar.gz', 'Icon\r', 'model04.tar.gz', 'model01.tar.gz', 'model03.tar.gz', 'bidaf-model-2017.09.15-charpad.tar.gz']
